In [2]:
# Import dependencies

import os
import sys
sys.path.insert(0, '..')

import pandas as pd
import geopandas as gpd
import numpy as np
import rioxarray
import xarray as xr
from ipyleaflet import Map, LayersControl, basemaps

import depal as dep
from model import add_image_values, get_model_prediction, get_overlay

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# datacube

# from datacube.utils.dask import start_local_dask
# from dep_tools.loaders import Sentinel2OdcLoader
# from dep_tools.namers import LocalPath
# from dep_tools.writers import LocalDsWriter

# from src.run_task import GeoMADSentinel2Processor, get_grid

In [3]:
dep.init()

Initiating DEPAL...
<Client: 'tcp://127.0.0.1:39373' processes=4 threads=8, memory=62.79 GiB>
/user/nicholasm@spc.int/proxy/8787/status


#### 1) Import training image.tif - read in .tif with the year as the name of the variable

var year ='2019';  
print('Year: '+year);

In [ ]:
# Define a training image.tif as a year then read in .tif with the year as the name of the variable (eg. 2019)
year = 2019

training_data = gpd.read_file("lulc_combined.geojson")


# Print the year of analysis
print("Year:", year)

#### 2) Add training data 
Map.addLayer(table4)  
var roi = VitiLevu;  // Assuming VitiLevu is defined somewhere  
//var year = '2022'  
Map.addLayer(roi, {}, 'ROI');  


In [6]:
# 2) Add training data 

training_data = gpd.read_file("training_data.geojson")



In [ ]:
# Define area of interest
aoi = dep.get_country_boundary("Fiji")

# Define training image
training_image = dep.get_landcover_mosaic(aoi, year, resolution=10, coastal_clip=False)


#### 3) Additional data preprocessing


Map.addLayer(buildings)  
var place = 'VitiLevu';  
var toClassify = image12;  // Assuming image2 is defined somewhere  
var ndviThreshold = 0.3;  
var mndwiThreshold = 0.19  
var ndviMask = toClassify.select('NDVI').lte(ndviThreshold)  
var mndwiMask = toClassify.select('MNDWI').lte(mndwiThreshold)  

In [7]:
# ML Component

from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict

model = RandomForestClassifier(max_depth=4, class_weight="balanced")
#model = GradientBoostingClassifier()
#model = HistGradientBoostingClassifier() #For coastal clipped mosaic

# Convert the dataframe data to numpy arrays
y = d.LULC_code.to_numpy()
bands = ["blue", "green", "red", "rededge", "nir"]
X = d.loc[:, bands].to_numpy()

# A coarse way to assess model fit, this returns a list of loss metric values 
# (particular metric depends on classifier) for a 10-fold cross validation. 
# Alternatively you could split into train and test (and validation) sets, 
#but for this few number of points this may be the best approach.
cv = cross_val_score(model, X, y, cv=10)
cv.mean()

In [ ]:
# Input from Sachin on NDVI masking of 0.3

masked = ndvi.where(gmw.squeeze())
# mangroves = xr.where(masked > 0.4, 1, np.nan)
# regular_mangroves = mangroves.where(masked <= 0.7)
# closed_mangroves = mangroves.where(masked > 0.7)
 
nvdi (dataarray) > 0.3

In [11]:
# Replicated for MNDWI

# masked = mndwi.where(gmw.squeeze())

 
# mndwi (dataarray) > 0.19


// Assuming 'yourFeatureCollection' is your original FeatureCollection that you want to modify var yourFeatureCollection; // Initialize this with your actual FeatureCollection

// Update the 'presence' property of all features to 0 var updatedFeatureCollection = NonExtraction.map(setPresenceToZero);

// Assuming 'table' is your original FeatureCollection var table; // Initialize this with your actual FeatureCollection

// Merge the updatedFeatureCollection with the original 'table' var mergedFeatureCollection = table.merge(updatedFeatureCollection).merge(Extraction);

// Now, 'mergedFeatureCollection' contains features from both 'table' and 'updatedFeatureCollection'

var bands = NDmedian.bandNames();

// Sample images using training samples var samples = NDmedian.sampleRegions({ collection: mergedFeatureCollection, // Assuming table is defined somewhere properties: ['presence'], scale: 10 }).randomColumn('random');

var split = 0.80; var training = samples.filter(ee.Filter.lt('random', split)); var testing = samples.filter(ee.Filter.gte('random', split));

// Initialize Maxent classifier parameters var maxentParams = { outputFormat: 'cloglog', autoFeature: true, betaMultiplier: 1, extrapolate: true, doClamp: true, writeClampGrid: true };

// Initialize and train the classifier // ... (the rest of your code remains unchanged)

// Initialize and train the classifier /* var classifier = ee.Classifier.amnhMaxent(maxentParams).train({ features: training,//.select([ 'VV','VH', 'NDMI','MNDWI','NDVI','SoilBarenessIndex', 'presence']), classProperty: 'presence', inputProperties: bands//[ 'VH','VV', 'NDVI','NDMI','MNDWI','SoilBarenessIndex']//bands }); */ var classifier = ee.Classifier.amnhMaxent({ outputFormat: 'cloglog', autoFeature: false, betaMultiplier: 1.2, extrapolate: false, doClamp: true, writeClampGrid: false, randomTestPoints: 10, seed: 42 }).train({ features: training.select([ 'R1', 'R2', 'R3', 'R4', 'R5', 'R7', 'R8', 'R9', 'R10', 'R11', 'R12', 'R13', 'R14', 'R15', 'R54', 'R35', 'R6', 'MNDWI','SAVI','EVI','NDMI', 'NDVI','SR', 'VV', 'VH','VV_1','VH_1','SoilBarenessIndex','presence']),//[ 'SAVI', 'EVI', 'MNDWI', 'NDMI', 'NDVI','VV','VH','VV_1', 'VH_1','presence']),//['R1', 'R2', 'R3', 'R4', 'R6', 'R7', 'R10', 'R11', 'R12', 'R13', 'R14', 'R15','VH','SoilBarenessIndex','SR', 'R54','R35','presence']), classProperty: 'presence', inputProperties: ['R1', 'R2', 'R3', 'R4', 'R5', 'R7', 'R8', 'R9', 'R10', 'R11', 'R12', 'R13', 'R14', 'R15', 'R54', 'R35', 'R6', 'MNDWI','SAVI','EVI', 'NDMI', 'NDVI','SR', 'VV', 'VH','VV_1','SoilBarenessIndex','VH_1']//[ 'SAVI', 'EVI','MNDWI', 'NDMI', 'NDVI','VV','VH','VV_1','VH_1']//['R1', 'R2', 'R3', 'R4', 'R6', 'R7', 'R10', 'R11', 'R12', 'R13', 'R14', 'R15','VH', 'SoilBarenessIndex','SR', 'R54', 'R35']//bands }); //'R1', 'R2', 'R3', 'R4', 'R5', 'R7', 'R8', 'R9', 'R10', 'R11', 'R12', 'R13', 'R14', 'R15', 'R54', 'R35', 'R6', //'R1', 'R2', 'R3', 'R4', 'R5', 'R7', 'R8', 'R9', 'R10', 'R11', 'R12', 'R13', 'R14', 'R15', 'R54', 'R35', 'R6', //'MNDWI','SAVI','EVI','EVI', 'NDMI','NDMI', 'NDVI','SR', 'SR','SoilBarenessIndex', var classified = toClassify.select(bands).classify(classifier);

var threshold = 0; // Set your probability threshold here classified = classified//.focal_mean({kernel: ee.Kernel.square({radius: 1}), iterations: 2}); var buildingMask = classified.clip(buildings).mask().rename('building_mask'); var roadMask = classified.clip(roads).mask().rename('road_mask');

// Invert the masks to keep pixels inside buildings and roads var invertedBuildingMask = buildingMask.not(); var invertedRoadMask = roadMask.not();

// Combine the inverted masks using logical AND to mask out pixels inside buildings or roads var combinedMask = invertedBuildingMask.and(invertedRoadMask);

// Update the river gravel image using the combined mask to mask out unwanted pixels var maskedClassified = classified.updateMask(combinedMask).select('probability').updateMask(classified.select('probability').gte(threshold));

// Add the probability band of the masked classified image to the map.['blue', 'green', 'yellow', 'red'] Map.addLayer(maskedClassified, imageVisParam2, 'MaxEnt Classification Probability above Threshold');

Export.image.toAsset({ image: maskedClassified, description: year[0]+'opt_sar'+place+'_clipped', assetId: place+'/NDSV/'+year[0]+'opt_sar'+place+'_clipped', region: roi,//.geometry().bounds(), scale: 10, maxPixels: 1e13 });

In [9]:
# var threshold = 0; // Set your probability threshold here classified = classified//.focal_mean({kernel: ee.Kernel.square({radius: 1}), iterations: 2}); var buildingMask = classified.clip(buildings).mask().rename('building_mask'); var roadMask = classified.clip(roads).mask().rename('road_mask');

# // Invert the masks to keep pixels inside buildings and roads var invertedBuildingMask = buildingMask.not(); var invertedRoadMask = roadMask.not();

# // Combine the inverted masks using logical AND to mask out pixels inside buildings or roads var combinedMask = invertedBuildingMask.and(invertedRoadMask);

# // Update the river gravel image using the combined mask to mask out unwanted pixels var maskedClassified = classified.updateMask(combinedMask).select('probability').updateMask(classified.select('probability').gte(threshold));

# // Add the probability band of the masked classified image to the map.['blue', 'green', 'yellow', 'red'] Map.addLayer(maskedClassified, imageVisParam2, 'MaxEnt Classification Probability above Threshold');

# Export.image.toAsset({ image: maskedClassified, description: year[0]+'opt_sar'+place+'_clipped', assetId: place+'/NDSV/'+year[0]+'opt_sar'+place+'_clipped', region: roi,//.geometry().bounds(), scale: 10, maxPixels: 1e13 });
